In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display
import random
from fastai.vision import *

import math
import tensorflow as tf
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation,Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend, models
from sklearn.metrics import confusion_matrix

from tensorflow.keras.applications import VGG16, MobileNet
from keras.applications.vgg16 import preprocess_input
print("library loaded")

In [2]:
train_dir = "../input/cat-and-dog/training_set/training_set"
test_dir= "../input/cat-and-dog/test_set/test_set"

In [3]:
categories = os.listdir(train_dir)
print(str(len(categories)),'CATEGORIES are ', categories)

Category_count = len(categories);

In [4]:
image = load_img("../input/cat-and-dog/training_set/training_set/dogs/dog.1005.jpg")
plt.imshow(image)
plt.axis("off")
plt.show()

imagedata = img_to_array(image)
shape = imagedata.shape
print('Figures are ', shape)

In [5]:
datagen = ImageDataGenerator(rescale=1./255, )

In [6]:
train_data =datagen.flow_from_directory(train_dir, target_size=(224,224))
test_data = datagen.flow_from_directory(test_dir, target_size=(224,224))

In [7]:
train_data.class_indices

In [8]:
Train_groups = len(train_data)
Test_groups = len(test_data)

In [9]:
imgs, labels = next(train_data)

In [10]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(30,30))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [11]:
plotImages(imgs)
print(labels)

In [12]:
model = Sequential()


model.add(Conv2D(64, (3, 3), padding='same',input_shape=(224,224,3))) 
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization()) 

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.35)) #64 --> 42

model.add(Conv2D(64, (3, 3), padding='same')) 
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Flatten()) 
model.add(Dropout(0.5)) 
model.add(Dense(512)) 
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(2)) 
model.add(Activation('softmax'))

model.summary()

In [13]:
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

In [14]:
Augment_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    zoom_range=0.2,  
    horizontal_flip=True,
    fill_mode='nearest') 
Augmentation_train = Augment_datagen.flow_from_directory(train_dir, target_size=(224,224))

In [15]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
history = model.fit_generator( 
    Augmentation_train, 
    epochs = 20,
    validation_data = test_data, verbose = 2, callbacks=[es]
    )

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot loss values vs epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(test_data, verbose=1)


In [ ]:
test_imgs, test_labels = next(test_data)


In [ ]:
plotImages(test_imgs)
print(test_labels)

In [ ]:
predictions = model.predict(x=test_data, verbose=0)